# <span class="tema">(Numèrics)</span> El nombre primer que ocupa la posició 10001

<p style="font-family:Arial;font-size:1em">
Si llistem els primers 6 nombres primers: 2, 3, 5, 7, 11, i 13, podem veure que el 6è primer és el 13. Quin és el primer que ocupa la posició 10001?
</p>

### Conceptualització problema

1. Quin algorisme hem d'aplicar? Revisa els algorismes vistos a teoria
1. Explica els passos que fa l'algorisme en el cas primer(6)

### Explicació de l'algortime emprat

#### `nth_prime_upper_bound(n)`
Retorna una fita superior del $n$-èssim número primer. [Referència](https://stackoverflow.com/questions/1042717/is-there-a-way-to-find-the-approximate-value-of-the-nth-prime#25440642).

#### `sieve(n)`
Computa una fita superior del $n$-èssim primer utilitzant `upper_n = nth_prime_upper_bound(n)`.
Després utilitza l'algoritme sieve of Eratosthenes per retornar un generator amb tots els primers menors a `upper_bound`.

#### `primer(n)`
Donada un generator generat per `sieve(n)`, retorna l'element en la $n-1$ posició d'aquest. S'utilitza la funció `next` per a iterar sobre un generator.

### Memoització

#### `nth_prime_upper_bound(n)` 
Es cacheja els resultats en forma de diccionari normal.

#### `sieve(n)`
Es realitza un cache especial per a generators. D'aquesta manera, si tenim guardat un generator per a calcular el primer $n$-èssim, i ens demanen el primer $m$-èssim (tal que $m < n$), llavors s'agafa el generator en cache. 

Per a poder realitzar aquest tipus de cache, s'utilitza la funció `itertools.tee`.


### Implementació

In [1]:
from time import time
from math import sqrt, log, ceil
from random import randint

In [2]:
def memo_bound(f):
    memo = {}
    def helper(x):
        if x not in memo:
            memo[x] = f(x)
        return memo[x]
    return helper

@memo_bound
def nth_prime_upper_bound(n):
    primes_small = [0, 2, 3, 5, 7, 11]

    fn = n
    if n < 6:
        return primes_small[n]
    
    flogn  = log(n)
    flog2n = log(flogn)

    if n >= 688383:         # Dusart 2010 page 2
        upper = fn * (flogn + flog2n - 1.0 + ((flog2n-2.00)/flogn))

    elif n >= 178974:       # Dusart 2010 page 7
        upper = fn * (flogn + flog2n - 1.0 + ((flog2n-1.95)/flogn));

    elif n >=  39017:       # Dusart 1999 page 14
        upper = fn * (flogn + flog2n - 0.9484);

    else:                   # Modified from Robin 1983 for 6-39016
        upper = fn * (flogn  +  0.6000 * flog2n);

    if upper >= 18446744073709551615:
        if n <= 425656284035217743:
            return 18446744073709551557
    
        exit(1)

    return ceil(upper)

In [3]:
from itertools import tee
from types import GeneratorType

Tee = tee([], 1)[0].__class__

def memo_sieve(f):
    memo = {}
    
    def helper(x):
        if x not in memo:
            memo[x] = f(x)
        
        if isinstance(memo[x], (GeneratorType, Tee)):
            memo[x], r = tee(memo[x])
            return r
    
        return memo[x]
    
    return helper

@memo_sieve
def sieve(n):
    upper_n = nth_prime_upper_bound(n)
    
    sieve = [True for _ in range(2, upper_n + 1)]

    for j in range(2, ceil(sqrt(upper_n))):
        i = j-2
        
        if sieve[i]:
            for k in range(j * j, upper_n + 1, j):
                sieve[k-2] = False

    return (j for j in range(2, upper_n+1) if sieve[j-2])

In [4]:
def primer(n):
    return next((x for i, x in enumerate(sieve(n)) if i == n-1), None)

In [5]:
def pretty_primer(n):
    return f"El {n}-èssim primer és {primer(n)}."

In [6]:
pretty_primer(1001)

'El 1001-èssim primer és 7927.'

In [7]:
N_TESTS = 10**4
MIN, MAX = 10**1, 10**3

t_total = 0
t_max, t_min = 0, 18446744073709551615

for _ in range(N_TESTS):
    n = randint(MIN, MAX)
    
    t = time()
    p = primer(n)  
    t = time() - t
    
    t_max = t if t > t_max else t_max
    t_min = t if t < t_min else t_min
    t_total += t

print(f"Elapsed time for {N_TESTS} computations: {t_total:4.6} s.")
print(f"Mean: {t_total/N_TESTS:2.6} s.")
print(f"Min:  {t_min:2.6} s.")
print(f"Max:  {t_max:2.6} s.")

Elapsed time for 10000 computations: 1.26958 s.
Mean: 0.000126958 s.
Min:  1.90735e-06 s.
Max:  0.01315 s.


```
Elapsed time for 10000 computations: 1.26889 s.
Mean: 0.000126889 s.
Min:  2.14577e-06 s.
Max:  0.022805 s.
```

### Avaluació (0 a 10 punts)


Concepte | Puntuació 
--- | --- 
Solució correcta eficient | **8** punts
Ús correcte algorismes teoria | **+1** punt
Solució correcta ineficient | **3** punts 
Codi comentat i seguint estàndar PEP8 | **+1** punt 
S'ofereix una funció adicional per mostrar la solució elegantment| **+0.5** punts 
L'algorisme falla repetidament | **-7** punts 
L'algorisme falla en algun cas excepcional | **-2** punt
No es donen prous exemples d'execució | **-1** punt
Codi, noms de variables, solució o comentaris no prou clars | **-1** punt
La funció o els paràmetres no s'anomenen com a l'exemple | **-1** punt